In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score
import shap  # SHAP分析
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import RBF
from catboost import CatBoostRegressor
## 为了正确评估模型性能，将数据划分为训练集和测试集，并在训练集上训练模型，在测试集上验证模型性能。
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 或者其他中文字体，确保字体文件存在
matplotlib.rcParams['axes.unicode_minus'] = False  # 防止负号显示为方块
import os
import shutil
from tqdm import tqdm
import tqdm
import logging
import pickle
# 设置日志配置，将输出写入文件
logging.basicConfig(filename='globa_model_training.log', level=logging.INFO, format='%(asctime)s - %(message)s', filemode='w')

def clear_folder(folder_path):
    # 确保文件夹存在
    if os.path.exists(folder_path):
        # 遍历文件夹中的所有文件和子文件夹
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            
            # 如果是文件，则删除
            if os.path.isfile(file_path):
                os.remove(file_path)
            # 如果是文件夹，则递归删除
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
    else:
        print(f"文件夹 {folder_path} 不存在！")
        # 创建文件夹
        os.makedirs(folder_path)

def standardize_data(x_train, x_test):
    scaler = StandardScaler()
    
    # 对训练数据进行拟合并转换
    x_train_scaled = scaler.fit_transform(x_train)
    
    # 对测试数据进行转换（使用训练数据的均值和标准差进行转换）
    x_test_scaled = scaler.transform(x_test)
    
    return x_train_scaled, x_test_scaled


In [2]:
date = '0624'

In [3]:
def plot_model_predictions(y_true, y_pred, title=None):
    years = list(range(1961, 2023))  # 从1961年到2022年

    plt.figure(figsize=(12,6))  # 调整图形的大小
    # 更改配色：使用深蓝色和橙色，给线条增加标记点
    plt.plot(years, y_true, color='#1f77b4', label='True Value', linewidth=2)
    plt.plot(years, y_pred, color='#ff7f0e', label='Predicted Value', linewidth=2, linestyle='--')  # 使用虚线
    plt.legend(loc='best', fontsize=12)  # 增加图例字体大小
    
    if title:
        plt.title(title, fontsize=16, fontweight='bold')  # 设置标题的字体大小和粗体
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Ammonia Emission', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)  # 添加网格线，增加可读性
    plt.tight_layout()  # 自动调整子图参数，使图形更紧凑

    # 保存图像而不是显示图像
    plt.savefig(f'../0311_pred_result/{title}.png', dpi=300)  # 保存为 PNG 格式，分辨率为300
    plt.close()

# 评估模型性能
def evaluate_model(model, x_train_scaled, y_train, x_test_scaled, y_test):
    
    if isinstance(model, CatBoostRegressor):
        model.fit(x_train_scaled, y_train, eval_set=(x_test_scaled, y_test), use_best_model=True)
    else:
        model.fit(x_train_scaled, y_train)
    y_pred_train = model.predict(x_train_scaled)
    y_pred_test = model.predict(x_test_scaled)
    
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    
    # print(f"Train R²: {train_r2:.4f}")
    # print(f"Test R²: {test_r2:.4f}")
    return train_r2 , test_r2

# SHAP分析
def shap_analysis(model, X, name=None, country=None):
    # 判断模型类型，选择相应的解释器
    if isinstance(model, LinearRegression):
        explainer = shap.LinearExplainer(model, X)
    elif isinstance(model, (SVR, GaussianProcessRegressor)):
        explainer = shap.KernelExplainer(model.predict, X)
    elif isinstance(model, RandomForestRegressor) or isinstance(model, xgb.XGBRegressor):
        explainer = shap.TreeExplainer(model)
    elif isinstance(model, CatBoostRegressor):
        explainer = shap.TreeExplainer(model)
    else:
        raise ValueError(f"Unsupported model type: {type(model)}")

    # 计算SHAP值
    shap_values = explainer.shap_values(X)

    # 可视化SHAP值
    shap.summary_plot(shap_values, X, plot_type="dot", show=False)
    
    # 保存shap图
    if country is not None:
        plt.savefig(f'../{date}_shap_result/{country}_{name}_shap_summary_plot.png')
    else:
        plt.savefig(f'../{date}_shap_result/{name}_shap_summary_plot.png')
    
    # 关闭图形，避免后续图形显示问题
    plt.close()

    # 计算每个特征的平均绝对SHAP值
    feature_importance = pd.DataFrame()
    feature_importance['特征'] = X.columns
    
    # 如果shap_values是一维数组（对于回归问题）
    if isinstance(shap_values, np.ndarray) and len(shap_values.shape) == 2:
        # 计算每个特征的平均绝对SHAP值
        feature_importance['平均绝对SHAP值'] = np.abs(shap_values).mean(axis=0)
        # 计算每个特征的平均SHAP值（可能为正或负）
        feature_importance['平均SHAP值'] = shap_values.mean(axis=0)
        feature_importance['归一化平均绝对SHAP值'] = feature_importance['平均绝对SHAP值'] / np.sum(feature_importance['平均绝对SHAP值'])
    
    # 按重要性排序
    feature_importance = feature_importance.sort_values('归一化平均绝对SHAP值', ascending=False)
    
    # 保存特征重要性到CSV文件
    output_dir = f'../{date}_shap_result'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 保存详细的SHAP值（每个样本的每个特征的SHAP值）
    detailed_shap_values = pd.DataFrame(shap_values, columns=X.columns)
    detailed_shap_values.to_csv(f'{output_dir}/{country}_{name}_detailed_shap_values.csv', index=False)
    
    # 保存特征重要性摘要
    feature_importance.to_csv(f'{output_dir}/{country}_{name}_feature_importance.csv', index=False)
    
    # 打印特征重要性
    print(f"\n特征重要性排名 (模型: {name}, 国家: {country}):")
    print(feature_importance)

# 模型选择
models = {
    'SVR': SVR(
        kernel='rbf',
        degree=3,
        tol=0.001
    ),
    'XGBoost': xgb.XGBRegressor(
        max_depth=3,               # 降低树的深度
        learning_rate=0.05,        # 降低学习率
        n_estimators=500,          # 增加迭代次数来弥补学习率降低的效果
        n_jobs=4,
        colsample_bytree=0.6,      # 调整特征采样比例
        subsample=0.7,             # 调整训练样本的采样比例
        min_child_weight=3,        # 增加每个叶子节点的最小权重
        alpha=1,                   # 增加L1正则化
        random_state=32
    ),  
    'LR': LinearRegression(
        fit_intercept=True,
        n_jobs=1
    ),
    # 'GPR': GaussianProcessRegressor(
    #     kernel=RBF(length_scale=1.0)
    # ),
    'RandomForest': RandomForestRegressor(
        n_estimators=200,
        criterion='squared_error',
        max_depth=15,
        min_samples_split=2,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_features=5,
        max_leaf_nodes=None,
        bootstrap=True,
        oob_score=False,
        n_jobs=1,
        random_state=None,
        verbose=0,
        warm_start=False
    ),
    'svr': SVR(
        kernel='rbf',
        degree=3,
        coef0=0.0,
        tol=0.001,
        C=1.0,
        epsilon=0.1,
        shrinking=True,
        cache_size=200,
        verbose=False,
        max_iter=-1
    ),
    'catboost': CatBoostRegressor(
        iterations=1000,
        early_stopping_rounds=200,
        learning_rate=0.05,
        loss_function="RMSE",
        eval_metric="RMSE",
        depth=6,
        min_data_in_leaf=20,
        random_seed=42,
        logging_level='Silent',
        use_best_model=True,
        one_hot_max_size=5,
        boosting_type="Ordered",
        max_ctr_complexity=2,
        nan_mode='Min'
    )
}

In [5]:
file_path = '国家氨排放强度原因解析_finalprotein.xlsx'
file_path_old = '副本国家氨排放强度原因解析V30416.xlsx'
df = pd.read_excel(file_path)
df_old = pd.read_excel(file_path_old)
# date = '0422'

clear_folder(f'../{date}_pred_result')
clear_folder(f'../{date}_shap_result')

文件夹 ../0624_pred_result 不存在！
文件夹 ../0624_shap_result 不存在！


In [ ]:
df_old

,大洲,area,year,areayear,Fertilizer N input intensity,The proportion of BNF in the total N input,Vegetable and fruit land share,Grassland area share,Livestock protein share,The proportion of the number of layer,The proportion of the number of meat cattle,The proportion of the number of meat chicken,The proportion of the number of dairy,The proportion of the number of sheep,GDP,Livestock NUE,Crop NUE,The proportion of manure fertilizer,Ammonia intensity
0,Asia,Afghanistan,1961,Afghanistan1961,0.000131,0.069490,0.047887,0.794702,0.090492,0.017874,0.034472,0.015640,0.223428,0.708586,NaN,0.016217,0.687050,28.122601,0.130973
1,Asia,Afghanistan,1962,Afghanistan1962,0.000130,0.068705,0.046264,0.793651,0.091496,0.019537,0.036536,0.016651,0.222009,0.705267,NaN,0.016645,0.680001,28.523233,0.133775
2,Asia,Afghanistan,1963,Afghanistan1963,0.000129,0.067865,0.045832,0.792602,0.106468,0.019833,0.036586,0.016599,0.240214,0.686767,NaN,0.017499,0.692667,29.297122,0.150929
3,Asia,Afghanistan,1964,Afghanistan1964,0.000071,0.069574,0.048010,0.791452,0.100537,0.020527,0.036838,0.017106,0.238255,0.687275,NaN,0.017755,0.727601,54.369007,0.136914
4,Asia,Afghanistan,1965,Afghanistan1965,0.000071,0.068764,0.051077,0.791348,0.104590,0.021382,0.035950,0.017476,0.255529,0.669663,NaN,0.018486,0.723083,56.018675,0.137684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10783,Africa,Zimbabwe,2018,Zimbabwe2018,0.013453,0.260781,0.034954,0.810788,0.332854,0.031714,0.278691,0.127887,0.343577,0.193321,1464.904683,0.057702,0.529415,0.164170,0.229982
10784,Africa,Zimbabwe,2019,Zimbabwe2019,0.012224,0.229063,0.048898,0.820466,0.437195,0.029455,0.245166,0.174000,0.321075,0.211223,1348.452786,0.062559,0.396521,0.175184,0.267709
10785,Africa,Zimbabwe,2020,Zimbabwe2020,0.012660,0.233394,0.055069,0.815258,0.336330,0.034020,0.235557,0.170546,0.322363,0.209912,1251.309431,0.059986,0.544224,0.181769,0.220883
10786,Africa,Zimbabwe,2021,Zimbabwe2021,0.016138,0.229331,0.047907,0.822878,0.326809,0.028785,0.245962,0.184607,0.313952,0.200147,1303.207454,NaN,NaN,0.147741,0.196094


In [6]:
cols = df_old.columns
quadrant_cols = df['区分象限']
df = df[cols]

In [7]:
df.iloc[10445]
# 删除df.iloc[10445]和5237          
df = df.drop(df.index[10445])
df = df.drop(df.index[5237])

# 数据补全

In [8]:
def interpolate_missing_by_area(df, area_col='area', year_col='year', value_col='value', model='SVR'):
    """
    对每个area分组，使用线性回归模型插值每个国家的前几年空值。
    
    Parameters:
    df (pd.DataFrame): 输入数据，包含area, country, year, value列
    area_col (str): 区域列名
    country_col (str): 国家列名
    year_col (str): 年份列名
    value_col (str): 需要插值的数值列名
    
    Returns:
    pd.DataFrame: 插值后的数据框
    """
    # 复制数据框以避免修改原始数据
    result_df = df.copy()
    
    # 获取所有唯一区域
    areas = result_df[area_col].unique()
    
    for area in areas:
        # 提取当前区域的数据
        area_data = result_df[result_df[area_col] == area]
        
        # 分离非空和空值行
        non_null_data = area_data[area_data[value_col].notnull()]
        null_data = area_data[area_data[value_col].isnull()]
        
        # 如果有足够的非空数据进行训练
        if len(non_null_data) >= 2:
            # 准备训练数据
            X_train = non_null_data[year_col].values.reshape(-1, 1)
            y_train = non_null_data[value_col].values
            
            # 数据标准化
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)

            if model == 'LinearRegression':
                # 训练线性回归模型
                model = LinearRegression()
                model.fit(X_train, y_train)
            elif model == 'SVR':
                # 训练非线性回归模型
                model = SVR(kernel='rbf', degree=3, tol=0.001)
                model.fit(X_train, y_train)
            elif model == 'XGBoost':
                # 训练XGBoost模型
                model = xgb.XGBRegressor(
                    max_depth=3,
                    learning_rate=0.05,
                    n_estimators=500,
                    n_jobs=4,
                    colsample_bytree=0.6,
                    subsample=0.7,
                    min_child_weight=3,
                    alpha=1,
                    random_state=32
                )
            # 预测空值
            if not null_data.empty:
                X_pred = null_data[year_col].values.reshape(-1, 1)
                # 数据标准化
                X_pred = scaler.transform(X_pred)
                predicted_values = model.predict(X_pred)
                
                # 将预测值填回结果数据框
                result_df.loc[(result_df[area_col] == area) & 
                            (result_df[value_col].isnull()), 
                            value_col] = predicted_values

    return result_df

In [9]:
print(df.isnull().sum())

大洲                                                 0
area                                               0
year                                               0
areayear                                           0
Fertilizer N input intensity                       0
The proportion of BNF in the total N input       924
Vegetable and fruit land share                     0
Grassland area share                               0
Livestock protein share                            0
The proportion of the number of layer              0
The proportion of the number of meat cattle        0
The proportion of the number of meat chicken       0
The proportion of the number of dairy              0
The proportion of the number of sheep              0
GDP                                             1566
Livestock NUE                                    348
Crop NUE                                         348
The proportion of manure fertilizer              624
Ammonia intensity                             

In [10]:
# 删除Livestock NUE，Crop NUE为0的行
df = df[df['Livestock NUE'] != 0]
df = df[df['Crop NUE'] != 0]
# 删除列名为gdp的列
df = df.drop(columns=['GDP'])

In [11]:
# 线性插值
for column in (df.columns.unique()[df.isnull().sum() > 0]).tolist():
    df[column] = df.groupby('area')[column].transform(lambda group: group.interpolate())
# 删除year in [2021, 2022]
df = df[~df['year'].isin([2021, 2022])]

In [12]:
print(df.isnull().sum())

大洲                                                0
area                                              0
year                                              0
areayear                                          0
Fertilizer N input intensity                      0
The proportion of BNF in the total N input        0
Vegetable and fruit land share                    0
Grassland area share                              0
Livestock protein share                           0
The proportion of the number of layer             0
The proportion of the number of meat cattle       0
The proportion of the number of meat chicken      0
The proportion of the number of dairy             0
The proportion of the number of sheep             0
Livestock NUE                                     0
Crop NUE                                          0
The proportion of manure fertilizer             487
Ammonia intensity                                 0
dtype: int64


In [13]:
# df = interpolate_missing_by_area(df, value_col="GDP")
df = interpolate_missing_by_area(df, value_col="The proportion of BNF in the total N input")
# df = interpolate_missing_by_area(df, value_col="The proportion of manure fertilizer")


In [14]:
df[df['The proportion of manure fertilizer'].isnull()]['area'].unique() 

array(['Antigua and Barbuda', 'Barbados', 'Dominica', 'Grenada',
       'Kiribati', 'Malta', 'Mauritius', 'Saint Lucia', 'Seychelles'],
      dtype=object)

In [15]:
problem_area = df[df['The proportion of manure fertilizer'].isnull()]['area'].unique() 
df = df[~df['area'].isin(problem_area)]

In [16]:
# 删除Crop NUE 不属于[0, 1]的行
df = df[(df['Crop NUE'] >= 0) & (df['Crop NUE'] <= 1)]

In [17]:
def countryDataset(df):
    # # 去除year in [2021, 2022]
    # df = df[~df['year'].isin([2021, 2022])]
    # 去空
    df.dropna(inplace=True)
    # 去除不需要的列
    data = df.drop(['大洲', 'area', 'year', 'areayear', 'Ammonia intensity'], axis=1)
    # 划分特征和目标变量
    X_train = data
    y_train = df.loc[:, 'Ammonia intensity']
    if X_train.shape[0] == 0:
        raise Exception
    for area in df['area'].unique():
        year = len(df[df['area'] == area]['year'])
        step = int(year * 0.8)  
        indices = list(range(0, len(X_train), year))  # 以步长为year生成索引
        x_tra, x_val, y_tra, y_val = [], [], [], []

    # 根据索引分割数据
    for start in indices:
        end = min(start + step, len(X_train))  # 确保不超出数据长度
        x_tra.append(X_train[start:end])
        y_tra.append(y_train[start:end])
        if end < len(X_train):  # 如果不是最后一段数据，将其后面一部分作为验证集
            x_val.append(X_train[end:end + year - step])
            y_val.append(y_train[end:end + year - step])

    # 合并所有分割的部分
    x_tra = pd.concat(x_tra)
    y_tra = pd.concat(y_tra)
    x_val = pd.concat(x_val)
    y_val = pd.concat(y_val)

    return X_train, y_train, x_tra, x_val, y_tra, y_val

In [18]:
def global_plot_model_predictions(y_true, y_pred, title=None):
    # 使用数据的索引作为 X 坐标
    x_vals = range(len(y_true))  # 使用样本的索引作为 X 坐标

    plt.figure(figsize=(12,6))  # 调整图形的大小
    # 更改配色：使用深蓝色和橙色，给线条增加标记点
    plt.plot(x_vals, y_true, color='#1f77b4', label='True Value', linewidth=2)
    plt.plot(x_vals, y_pred, color='#ff7f0e', label='Predicted Value', linewidth=2, linestyle='--')  # 使用虚线
    plt.legend(loc='best', fontsize=12)  # 增加图例字体大小
    
    if title:
        plt.title(title, fontsize=16, fontweight='bold')  # 设置标题的字体大小和粗体
    plt.xlabel('Index', fontsize=14)  # 改为“Index”或其他你需要的标签
    plt.ylabel('Ammonia Emission', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)  # 添加网格线，增加可读性
    plt.tight_layout()  # 自动调整子图参数，使图形更紧凑

    # 保存图像而不是显示图像
    plt.savefig(f'../{date}_pred_result/{title}.png', dpi=300)  # 保存为 PNG 格式，分辨率为300
    plt.close()

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import os
from scipy.stats import gaussian_kde
from statsmodels.nonparametric.smoothers_lowess import lowess

# 设置matplotlib中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 300
plt.rcParams['axes.labelsize'] = 14  # 增大坐标轴标签
plt.rcParams['xtick.labelsize'] = 12  # 增大x轴刻度标签
plt.rcParams['ytick.labelsize'] = 12  # 增大y轴刻度标签
plt.rcParams['legend.fontsize'] = 12  # 增大图例字体
plt.rcParams['font.family'] = 'Arial'

def get_feature_category_colors(feature_names):
    """分配颜色给不同类别的特征"""
    # 按照要求定义类别
    categories = {
        'crop': ['Fertilizer N input intensity', 'The proportion of manure fertilizer', 
                'The proportion of BNF in the total N input', 'Vegetable and fruit land share',
                'Crop NUE'],
        'Grass': ['Grassland area share'],
        'livestock': ['Livestock protein share', 'The proportion of the number of layer', 
                    'The proportion of the number of meat cattle', 'The proportion of the number of meat chicken',
                    'The proportion of the number of dairy', 'The proportion of the number of sheep',
                    'Livestock NUE']
    }
    
    category_colors = {
        'crop': '#E69F00',      # 金黄
        'Grass': '#56B4E9',     # 天蓝
        'livestock': '#009E73'  # 青绿
    }
    
    # 为每个特征分配颜色
    feature_colors = {}
    for feature in feature_names:
        for category, features in categories.items():
            if feature in features:
                feature_colors[feature] = category_colors[category]
                break
        if feature not in feature_colors:
            feature_colors[feature] = '#9E9E9E'  # 默认灰色
    
    return feature_colors, category_colors, categories

def create_combined_shap_plot(shap_values, features, output_path='combined_shap_plot.png'):
    """
    创建组合SHAP图，类似于示例图的布局
    
    参数:
    shap_values: SHAP值数组
    features: 特征数据（DataFrame或numpy array）
    feature_names: 特征名称列表
    output_path: 输出文件路径
    """
    # 如果features是numpy数组，转换为DataFrame
    if isinstance(features, np.ndarray):
        features = pd.DataFrame(features, columns=feature_names)
    else:
        feature_names = features.columns
    # 计算每个特征的平均绝对SHAP值
    mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
    
    # 创建特征重要性DataFrame并排序
    importance_df = pd.DataFrame({
        '特征': feature_names,
        'SHAP值': mean_abs_shap
    })
    importance_df = importance_df.sort_values('SHAP值', ascending=True)  # 从小到大排序，绘图时会从下到上显示
    
    # 获取特征类别和颜色
    feature_colors, category_colors, categories = get_feature_category_colors(feature_names)
    
    # 创建图形：2行2列的不等大子图布局
    fig = plt.figure(figsize=(24, 12))
    
    # 添加子图字母标签
    plt.figtext(0.02, 0.98, 'a', fontsize=28, fontweight='bold', color='black', ha='left', va='top')
    plt.figtext(0.52, 0.98, 'b', fontsize=28, fontweight='bold', color='black', ha='left', va='top')
    plt.figtext(0.77, 0.98, 'c', fontsize=28, fontweight='bold', color='black', ha='left', va='top')
    plt.figtext(0.52, 0.48, 'd', fontsize=28, fontweight='bold', color='black', ha='left', va='top')
    plt.figtext(0.77, 0.48, 'e', fontsize=28, fontweight='bold', color='black', ha='left', va='top')
    
    # 子图1: 特征重要性条形图 (a) - 占据左半部分
    ax1 = plt.subplot2grid((2, 4), (0, 0), rowspan=2, colspan=2)
    
    # 按重要性排序的特征（取前15个）
    # 如果特征数量不足15个，则全部显示
    n_features = min(15, len(importance_df))
    top_features = importance_df.tail(n_features)  # 使用tail获取最大的几个值
    
    # 为每个特征分配颜色
    bar_colors = [feature_colors.get(feature, '#B0B0B0') for feature in top_features['特征']]
    
    # 绘制水平条形图
    bars = ax1.barh(range(len(top_features)), 
                   top_features['SHAP值'],
                   color=bar_colors,
                   height=0.7)
    
    # 设置y轴标签
    ax1.set_yticks(range(len(top_features)))
    ax1.set_yticklabels(top_features['特征'], fontsize=16) 

    # 设置x轴标签
    ax1.set_xlabel('Mean (|SHAP| value)', fontsize=16)   
    
    # 添加图例
    legend_handles = [plt.Rectangle((0,0),1,1, color=color) for color in category_colors.values()]
    legend_labels = list(category_colors.keys())
    ax1.legend(legend_handles, legend_labels, 
           loc='upper center', bbox_to_anchor=(0.5, -0.05),
           fancybox=True, shadow=False, ncol=4, fontsize=16, frameon=True)
    # 设置网格和边框
    ax1.grid(axis='x', linestyle='--', alpha=0.3)
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    
    # 从每个类别中选择最重要的特征
    top_category_features = []
    
    # 为每个类别找出最重要的特征
    for category, features_list in categories.items():
        # 过滤出当前类别的特征
        category_features = importance_df[importance_df['特征'].isin(features_list)]
        if not category_features.empty:
            if category == 'livestock':
                # 对于livestock类别，选择两个最重要的特征
                if len(category_features) >= 2:
                    top_category_features.extend(category_features.iloc[-2:]['特征'].tolist())
                else:
                    top_category_features.append(category_features.iloc[-1]['特征'])
            elif category == 'crop':
                top_category_features.append('Crop NUE')
                pass
            else:
                # 添加该类别中最重要的特征
                top_category_features.append(category_features.iloc[-1]['特征'])
    
    # 创建4个子图 - SHAP依赖图，排列在右侧
    subplot_positions = [(0, 2), (0, 3), (1, 2), (1, 3)]
    
    for i, feature in enumerate(top_category_features):
        if i >= len(subplot_positions):
            break
            
        row, col = subplot_positions[i]
        ax = plt.subplot2grid((2, 4), (row, col))
        
        # 获取该特征的索引
        feature_idx = list(feature_names).index(feature)
        
        # 获取特征的颜色
        color = feature_colors.get(feature, '#9E9E9E')
        
        # 绘制散点图
        scatter = ax.scatter(features[feature], shap_values[:, feature_idx], 
                  alpha=0.5, s=20, 
                  c=color)
        
        # 排序数据以便绘制平滑线
        idx = np.argsort(features[feature])
        sorted_x = features[feature].iloc[idx]
        sorted_y = shap_values[:, feature_idx][idx]
        
        # 使用LOWESS平滑
        try:
            if i == 3:
                smoothed = lowess(sorted_y, sorted_x, frac=0.3)
            else:
                smoothed = lowess(sorted_y, sorted_x, frac=0.6)
            # 绘制平滑线
            ax.plot(smoothed[:, 0], smoothed[:, 1], color='gray', linewidth=2)
        except:
            # 如果LOWESS失败，尝试简单移动平均
            window_size = max(5, len(sorted_y) // 20)  # 窗口大小
            rolling_mean = pd.Series(sorted_y).rolling(window=window_size, center=True).mean()
            ax.plot(sorted_x[rolling_mean.notna()], rolling_mean.dropna(), color='gray', linewidth=2)
        
        # 添加水平零线
        ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
        
        # 设置标签
        ax.set_xlabel(feature, fontsize=16)
        ax.set_ylabel('SHAP value', fontsize=16)
        
        # 设置轴范围
        y_max = max(abs(shap_values[:, feature_idx].min()), abs(shap_values[:, feature_idx].max()))
        ax.set_ylim(-y_max*1.1, y_max*1.1)
        
        # 设置网格和边框
        ax.grid(True, linestyle='--', alpha=0.1)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    
    # 调整布局
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # 留出顶部空间放字母标签
    
    # 保存图像
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    
    # 关闭图形
    plt.close()
    
    print(f"组合SHAP图已保存至 {output_path}")

In [20]:
def shap_drawing(X_df, best_model_patn, output_dir, period_name=None):
    if isinstance(best_model_patn, str):
        model = pickle.load(open(best_model_patn, 'rb'))
    else:
        model = best_model_patn

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_df)

    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 生成组合SHAP图
    output_path = f'{output_dir}/{period_name}_combined_shap_plot.png'
    create_combined_shap_plot(shap_values, X_df, output_path)



In [21]:
def train_model(df):
    # 训练模型并评估
    best_model = None
    best_model_name = None
    best_score = float('-inf')
    try:
        X, Y, x_tra, x_val, y_tra, y_val = countryDataset(df)
    except:
        raise Exception

    x_tra_scaled, x_val_scaled = standardize_data(x_tra, x_val) 
    X_scaled = np.concatenate((x_tra_scaled, x_val_scaled), axis=0)
    # 保存原始特征数据（未标准化的）
    X_original = np.concatenate((x_tra, x_val), axis=0)

    # 在模型评估中添加 tqdm 进度条
    for name, model in tqdm.tqdm(models.items(), desc=f"Training models", ncols=100):
        logging.info(f"Training {name}...")
        train_r2, test_r2 = evaluate_model(model, x_tra_scaled, y_tra, x_val_scaled, y_val)

        logging.info(f"Model: {name}, Test R2: {test_r2:.4f}")

        if test_r2 > best_score:
            best_model = model
            best_score = test_r2
            best_model_name = name

    logging.info(f"Best Model: {best_model_name} with Test R2: {best_score:.4f}")
    # 保存best_model
    if best_model is not None:
        if not os.path.exists(f'../{date}_model'):
            os.makedirs(f'../{date}_model')
        pickle.dump(best_model, open(f'../{date}_model/{best_model_name}.pkl', 'wb'))

    # 使用最佳模型进行预测
    # if best_model is not None:

    #     y_pred = best_model.predict(X_scaled)

        # 可视化预测结果
        # global_plot_model_predictions(Y, y_pred, title=f"{best_model_name} Ammonia Emission Prediction")

        # SHAP分析
        # shap_analysis(best_model, pd.DataFrame(X_scaled, columns=X.columns), best_model_name)

        # 确保所有日志输出已写入文件
        logging.info(f"Completed training and evaluation \n")
    return best_model, best_model_name, best_score, X_scaled, X_original

In [22]:
def shap_output(X_df, best_model_patn, output_dir, X_original=None):
    if isinstance(best_model_patn, str):
        model = pickle.load(open(best_model_patn, 'rb'))
    else:
        model = best_model_patn

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_df)

    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 生成组合SHAP图 - 使用原始特征值而不是标准化后的值
    output_path = f'{output_dir}/combined_shap_plot.png'
    # 如果提供了原始数据，使用原始数据；否则使用标准化后的数据
    features_for_plot = X_original if X_original is not None else X_df
    create_combined_shap_plot(shap_values, features_for_plot, output_path)

In [23]:
def shap_output(X_df, best_model_patn, output_dir, X_original=None):
    if isinstance(best_model_patn, str):
        model = pickle.load(open(best_model_patn, 'rb'))
    else:
        model = best_model_patn

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_df)

    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 生成组合SHAP图 - 使用原始特征值而不是标准化后的值
    output_path = f'{output_dir}/combined_shap_plot.png'
    # 如果提供了原始数据，使用原始数据；否则使用标准化后的数据
    features_for_plot = X_original if X_original is not None else X_df
    create_combined_shap_plot(shap_values, features_for_plot, output_path)

In [24]:
def shap_output(X_df, best_model_patn, output_dir, X_original=None):
    if isinstance(best_model_patn, str):
        model = pickle.load(open(best_model_patn, 'rb'))
    else:
        model = best_model_patn

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_df)

    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 生成组合SHAP图 - 使用原始特征值而不是标准化后的值
    output_path = f'{output_dir}/combined_shap_plot.png'
    # 如果提供了原始数据，使用原始数据；否则使用标准化后的数据
    features_for_plot = X_original if X_original is not None else X_df
    create_combined_shap_plot(shap_values, features_for_plot, output_path)

In [25]:
df_tags = {
    "all": df,
    # "before_1995": df[df['year'] < 1995],
    # "before_2017": df[df['year'] < 2017],
    # "after_2017": df[df['year'] >= 2017]
}

for tag, df_tag in df_tags.items():
    best_model, best_model_name, best_score, X_scaled, X_original = train_model(df_tag)
    # 使用原始数据作为特征值显示，但SHAP值仍基于标准化数据计算
    shap_output(pd.DataFrame(X_scaled, columns=df.columns[4:-1]), best_model, f"../{date}_shap_result/{tag}", 
                pd.DataFrame(X_original, columns=df.columns[4:-1]))

Training models: 100%|████████████████████████████████████████████████| 6/6 [00:10<00:00,  1.71s/it]


组合SHAP图已保存至 ../0624_shap_result/all/combined_shap_plot.png


# 四象限shap组合图

In [ ]:
# df_Quadrant = pd.read_excel('../Data/副本国家氨排放强度原因解析612.xlsx')
df_Quadrant = pd.concat([df, quadrant_cols], axis=1)

In [ ]:
Quadrants = df_Quadrant['区分象限'].unique()

for quadrant in Quadrants:
    if isinstance(quadrant, str):
        df_quadrant = df_Quadrant[df_Quadrant['区分象限'] == quadrant]
        df_quadrant = df_quadrant[df.columns]
        best_model, best_model_name, best_score, X_scaled, X_original = train_model(df_quadrant)
        # 使用原始数据作为特征值显示，但SHAP值仍基于标准化数据计算
        shap_output(pd.DataFrame(X_scaled, columns=df.columns[4:-1]), best_model, f"../{date}_shap_result/{quadrant}", 
                    pd.DataFrame(X_original, columns=df.columns[4:-1]))

Training models: 100%|████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.91s/it]


组合SHAP图已保存至 ../0613_shap_result/第二象限/combined_shap_plot.png


Training models: 100%|████████████████████████████████████████████████| 6/6 [00:10<00:00,  1.83s/it]


组合SHAP图已保存至 ../0613_shap_result/第三象限/combined_shap_plot.png


Training models: 100%|████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.46s/it]


组合SHAP图已保存至 ../0613_shap_result/第一象限/combined_shap_plot.png


Training models: 100%|████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.61s/it]


组合SHAP图已保存至 ../0613_shap_result/第四象限/combined_shap_plot.png
